## Guide to using RW to create tiled web maps

This document shows how to go from a bunch of Earth Engine (EE) images, to styled webmaps using the RW API.
You will need to have an authentication token for the RW API, and an EE account.


#### Useful links
* [RW API documentation](https://resource-watch.github.io/doc-api)
* [RW example notebooks repo](https://github.com/resource-watch/notebooks)


### Step 0. Check if the data is already in the API

It may be that the data you want to include is already in the API. You should check first to see if this is so.
E.g. You can see all of the datasets registerd for the `gfw` platform, for example with the below query:

http://api.resourcewatch.org/v1/dataset?app=gfw&page[size]=100000

Or, if you have an idea what the name of the dataset may be, you can search for it as follows:

http://api.resourcewatch.org/v1/dataset?app=gfw&name=viirs

If the dataset doesnt exist in the API, and you want to add it here are the steps to create a new dataset based on Earth Engine images.


### Step 1. Earth Engine (images -> image collection)
N.b. You can do the following via a script (e.g. see code [here](https://github.com/resource-watch/nrt-scripts/blob/master/utils/examples/GEE_upload_utils/src/__init__.py)), or via GUI interface on the EE JS editor.

**Using the editor:**

We will assume you have imported data into Earth Engine as a series of image assets.
The next step is to create an empty Image Collection.

![](./pics/empty_ic.png)

You will then need to select your collection from the list of `Assets`, and click copy an image into this collection:

![](./pics/copy_im_to_ic.png)

Then specify an image asset, and select a target name for the image (if different from the original).

![](./pics/add_im.png)

click the new image you have added to the collection. This should open a new modal window:

![](./pics/img_modal1.png)

Next, you will need to add Properties to the image, so that the RW API can query the image collection. Specifically you will need to add `system:time_start` with a value in the format of `YYYY-MM-DD`. To do this, Click `+ Add property`, and add the new info. Then click `Done`.

![](./pics/im_form_done.png)

Repeat this process for all images required in the collection. Again, alternativley you can do this via a script as shown [here](https://github.com/resource-watch/nrt-scripts/blob/master/utils/examples/GEE_upload_utils/src/__init__.py).

** N.b. You must ensure that the Image Collection Privacy is set to publicly accessible, via the `share` button on the Image Collection **

![](./pics/ic_share.png)

### Step 2. Register the dataset (EE image collection -> RW dataset)

The next step is to register the dataset in the Resource Watch API. This is done via HTTP requests as described in the RW documentation.

Register a dataset with a Post Request:

```bash
curl -X POST https://api.resourcewatch.org/v1/dataset \
-H "Authorization: Bearer <your-token>" \
-H "Content-Type: application/json"  -d \
'{
    "connectorType":"rest",
    "provider":"gee",
    "tableName": "projects/wri-datalab/Gain_carbon_test"
    "application":[
     "gfw"
    ],
    "name":"AGB Gains"
}'
```
You should recieve a response of the registered dataset info, note the `status` field may say pending on initial response:

```json
{
    "data": {
        "id": "b7a34457-1d8a-456e-af46-876e0b42fb96",
        "type": "dataset",
        "attributes": {
            "name": "AGB Gains",
            "slug": "AGB-Gains_1",
            "type": "raster",
            "subtitle": null,
            "application": [
                "gfw"
            ],
            "dataPath": null,
            "attributesPath": null,
            "connectorType": "rest",
            "provider": "gee",
            "userId": "5936af3802d342203186ab99",
            "connectorUrl": null,
            "tableName": "projects/wri-datalab/Gain_carbon_test",
            "status": "pending",
            "published": false,
            "overwrite": false,
            "verified": false,
            "blockchain": {},
            "mainDateField": null,
            "env": "production",
            "geoInfo": false,
            "protected": false,
            "legend": {
                "date": [],
                "region": [],
                "country": [],
                "nested": []
            },
            "clonedHost": {},
            "errorMessage": null,
            "taskId": null,
            "updatedAt": "2018-04-13T10:35:25.948Z",
            "widgetRelevantProps": [],
            "layerRelevantProps": []
        }
    }
}
```

You can use the ID (in this case `b7a34457-1d8a-456e-af46-876e0b42fb96`) together with the `dataset` endpoint to see the dataset info as needed and to check the `status` of the dataset. If successfully processed the status should read `"status": "saved"`.

https://api.resourcewatch.org/v1/dataset/b7a34457-1d8a-456e-af46-876e0b42fb96

Dataset objects cannot be visulised directly, since we want to make web map tiles from these data. To do that, we will need a final step, the creation of a **Layer** object in the API.

### Creating a layer (Dataset --> unique visulization)

To do this we will need to make another HTTP post to the API. We will need to include in it a way to iscolate an image from our colection, and to specifiy visulisation styles (using the [Styled Layer Descriptors - SLD - method in EE](https://developers.google.com/earth-engine/image_visualization)). More info on SLD can be found [here](http://docs.geoserver.org/stable/en/user/styling/sld/cookbook/).

You may want to test/experiment with the visulisation in the Earth Engine playground in advance of creating a new layer. Note that in lieu of masking the data, you can set opacity on certain values (which has the same final effect).



```bash
curl -X POST https://api.resourcewatch.org/v1/dataset/<dataset_id>/layer \
-H "Authorization: Bearer <your-token>" \
-H "Content-Type: application/json"  -d \
{"name":"test agb green 2018",
"application": [
"gfw"
],
"provider": "gee",
"layerConfig":{	
	"body":{
		"sldValue": "<RasterSymbolizer><ColorMap type=\"ramp\" extended=\"false\" ><ColorMapEntry color=\"#0D210F\" quantity=\"0\" opacity=\"0\" /><ColorMapEntry color=\"#0D210F\" quantity=\"1\" /><ColorMapEntry color=\"#77E285\" quantity=\"3000\" /></ColorMap></RasterSymbolizer>",
		"styleType": "sld"
		},
	"assetId": "projects/wri-datalab/Gain_carbon_test",
	"provider": "gee",
	"position": "last",
	"isImageCollection":true,
	"order": 2018,
	"filterDates": ["2008-01-01","2019-01-01"]
	
}
}

```

From which you will get back a registered layer response:

```json
{
    "data": {
        "id": "c9e48a9f-2dca-4233-9400-0b5e4e07674f",
        "type": "layer",
        "attributes": {
            "name": "test agb green 2018",
            "slug": "test-agb-green-2018",
            "dataset": "b7a34457-1d8a-456e-af46-876e0b42fb96",
            "application": [
                "gfw"
            ],
            "iso": [],
            "provider": "gee",
            "userId": "57a0aa1071e394dd32ffe137",
            "default": false,
            "protected": false,
            "env": "production",
            "layerConfig": {
                "body": {
                    "sldValue": "<RasterSymbolizer><ColorMap type=\"ramp\" extended=\"false\" ><ColorMapEntry color=\"#0D210F\" quantity=\"0\" opacity=\"0\" /><ColorMapEntry color=\"#0D210F\" quantity=\"1\" /><ColorMapEntry color=\"#77E285\" quantity=\"3000\" /></ColorMap></RasterSymbolizer>",
                    "styleType": "sld"
                },
                "assetId": "projects/wri-datalab/Gain_carbon_test",
                "provider": "gee",
                "position": "last",
                "isImageCollection": true,
                "order": 2018,
                "filterDates": [
                    "2008-01-01",
                    "2019-01-01"
                ]
            },
            "legendConfig": {},
            "interactionConfig": {},
            "applicationConfig": {},
            "staticImageConfig": {},
            "updatedAt": "2018-04-13T13:20:43.963Z"
        }
    }
}
```

Each time you want to create a different visulisation of a dataset you will need to register a new layer object.

Finally, you will be able to use the layers as a [tiled web map](https://en.wikipedia.org/wiki/Tiled_web_map), based on the layer `id`.

https://api.resourcewatch.org/v1/layer/c9e48a9f-2dca-4233-9400-0b5e4e07674f/tile/gee/{z}/{x}/{y}

In [3]:
import folium

In [8]:
url ="https://api.resourcewatch.org/v1/layer/c9e48a9f-2dca-4233-9400-0b5e4e07674f/tile/gee/{z}/{x}/{y}"

In [10]:
m = folium.Map(location=[-10.141931686131018, -55.283203125,], tiles='Open Street Map')
m.add_tile_layer(tiles=url, max_zoom=12, min_zoom=1,  attr='Custom tiles')

m

![](./pics/web_map_eg.png)